# Sakamoto BERT

## Preprocess

install Transformers

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.9MB/s 
     |████████████████████████████████| 1.1MB 28.3MB/s 
     |████████████████████████████████| 890kB 40.0MB/s 
     |████████████████████████████████| 2.9MB 50.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c1e5ca9c37a5891573fb82d151b6d7ac39dfe727517cb80fc023063015f1d2c3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


download the BERT Japanese pre-trained model

In [ ]:
!wget http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip

--2020-11-08 01:05:18--  http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip
Resolving nlp.ist.i.kyoto-u.ac.jp (nlp.ist.i.kyoto-u.ac.jp)... 133.242.249.182
Connecting to nlp.ist.i.kyoto-u.ac.jp (nlp.ist.i.kyoto-u.ac.jp)|133.242.249.182|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 411210545 (392M) [application/zip]
Saving to: ‘Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip’

Japanese_L-12_H-768 100%[===================>] 392.16M  18.8MB/s    in 22s     

2020-11-08 01:05:41 (17.5 MB/s) - ‘Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip’ saved [411210545/411210545]



In [ ]:
!unzip Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip -d bert

Archive:  Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers.zip
   creating: bert/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/
  inflating: bert/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/pytorch_model.bin  
  inflating: bert/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/vocab.txt  
  inflating: bert/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/config.json  
  inflating: bert/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/tokenizer_config.json  


copy Sakamoto SST-2

In [ ]:
!cp -r drive/My\ Drive/sakamoto-bert/SST-2 sakamoto_sst2

## Training

clone the repository

In [ ]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 50085 (delta 1), reused 1 (delta 0), pack-reused 50074
Receiving objects: 100% (50085/50085), 36.95 MiB | 27.42 MiB/s, done.
Resolving deltas: 100% (34957/34957), done.


In [ ]:
%cd transformers
!git checkout bb9559a7f98fc18747fd957c3bd2a6e4c1111e45
%cd ..

/content/transformers
Note: checking out 'bb9559a7f98fc18747fd957c3bd2a6e4c1111e45'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at bb9559a7 Don't use `store_xxx` on optional bools (#7786)
/content


fine-tune the model

In [ ]:
!python transformers/examples/text-classification/run_glue.py \
    --model_name_or_path bert/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers \
    --task_name SST-2 \
    --do_train \
    --do_eval \
    --data_dir sakamoto_sst2 \
    --max_seq_length 128 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --output_dir bert_sakamoto_sst2

2020-11-08 01:05:59.507562: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/08/2020 01:06:01 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/08/2020 01:06:01 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='bert_sakamoto_sst2', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov08_01-06-01_67b01f7980bb', loggin

copy the model

In [ ]:
!cp -r bert_sakamoto_sst2 drive/My\ Drive/sakamoto-bert/model

## Evaluation

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert_sakamoto_sst2')
model = AutoModelForSequenceClassification.from_pretrained('bert_sakamoto_sst2', return_dict=True)

In [ ]:
classes = ['not sksk_sskn', 'is sksk_sskn']

In [ ]:
import csv

sequences = []
with open('sakamoto_sst2/test.tsv', newline='') as f:
    reader = csv.DictReader(f, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        sequences.append(row['sentence'])

In [ ]:
for sequence in sequences:
    sequence_tensor = tokenizer(sequence, return_tensors='pt')
    classification_logits = model(**sequence_tensor).logits
    results = torch.softmax(classification_logits, dim=1).tolist()[0]

    print(sequence)
    for i in range(len(classes)):
        print(f'{classes[i]}: {results[i] * 100}%')
    print()

母 に ハンネ で 呼ば れた ので スマホ 食った
not sksk_sskn: 99.69808459281921%
is sksk_sskn: 0.30191761907190084%

午後 の 紅茶 が 美味し すぎて 口 の 形 が ペットボトル に なっちゃ った ……
not sksk_sskn: 44.24114227294922%
is sksk_sskn: 55.75885772705078%

ざむ ちゃん 実は 戦隊 モノ の 黄色 を 担当 して いる んだ よ ね
not sksk_sskn: 5.700467526912689%
is sksk_sskn: 94.29953098297119%

アタシ 、 大きく なったら ナン に なる の が 夢 だった んだ 、 母 が なり たくて も なれ なかった もの だ から …
not sksk_sskn: 98.44356775283813%
is sksk_sskn: 1.556432619690895%

来年 サカ ちゃん カレンダー 開催 し ます ので 覚悟 して おいて ください ね
not sksk_sskn: 99.71803426742554%
is sksk_sskn: 0.28196333441883326%

こんにちは 、 ” 坂本 ” … です … 　 ピキキ … （ メガネ が 割れる 音 ） 　 タスケテーッッッタスケテーッッッタスケテーッッッタスケテーッッッタスケテーッッッタスケテーッッッタスケテーッッッタスケテーッッッタスケテーッッッタスケテーッッッタスケテーッッッ 　 タスケテーッッッ 　 それ で は 、 さようなら
not sksk_sskn: 60.26184558868408%
is sksk_sskn: 39.73815739154816%

外 寒 すぎて デッケェ 冷え ピタ に 包ま れた とき と 全く 同じ 気持ち に なった
not sksk_sskn: 99.61721301078796%
is sksk_sskn: 0.38279134314507246%

口 に 入れた 梅干し が 想像 以上 に スッパ くて この世 が 一 点 に 圧縮 さ れた
not sksk_sskn: 98.9045917987823

# References

Transformers

- [transformers/examples/text-classification at master · huggingface/transformers](https://github.com/huggingface/transformers/tree/master/examples/text-classification)
- [Summary of the tasks — transformers 3.0.2 documentation](https://huggingface.co/transformers/task_summary.html#sequence-classification)

BERT Japanese Pretrained Model

- [BERT日本語Pretrainedモデル - KUROHASHI-CHU-MURAWAKI LAB](http://nlp.ist.i.kyoto-u.ac.jp/index.php?BERT%E6%97%A5%E6%9C%AC%E8%AA%9EPretrained%E3%83%A2%E3%83%87%E3%83%AB)

The Stanford Sentiment Treebank	

- [GLUE Benchmark](https://gluebenchmark.com/tasks)